In [ ]:
import warnings
warnings.filterwarnings('ignore')

# IM939 - Lab 3 - Regression Exercise ANSWERS

Here we use Scikit Learn library: https://www.tutorialspoint.com/scikit_learn/scikit_learn_linear_regression.htm


## Wine Dataset

The dataset for this task is the wine dataset. More information here: https://archive.ics.uci.edu/ml/datasets/wine+quality

_What would be your research question? What do you want to learn?_

(The wine dataset allows multiple different regression models. It is even prompted in the description of the dataset I linked above: "Also, we are not sure if all input variables are relevant. So it could be interesting to test feature selection methods." I will show one example here. Feel free to discuss your models in the channel on Teams and share it with you colleagues!)

My **research question** is: What determines the wine density?  
Therefore my dependent variable will be "density". What will be the predictors? I could do some quick research, look for some resources in order to create some hypotheses. This would be something like theory-based approach. 
What if we didn't do any research to help us build any hypotheses about wine (maybe besisdes from some tasting ;)? That's when we need to run a lot of checks and explore data more, looking for insights or patterns. I can just try all of them, or those that make more sense to me, given that they are only 11. This would be a data-driven approach. 

## Reading Data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Don't forget to change the path below to point to the winequality-red_v2.xlsx file
wine = pd.read_excel('/Users/u2272591/Downloads/WarwickCIM IM939_handbook main content-labs_Lab_3/data/raw/winequality-red_v2.xlsx')

#You might need to use encoding, then the code will look like:
# wine = pd.read_excel('winequality-red_v2.xlsx', encoding='UTF-8')

type(wine)

## Data exploration
### Univariate analysis
It is time to check the data, their distribution and central tendencies

In [ ]:
print('shape:', wine.shape)
wine.head()

Look at the quality column above. It is the only column where you don't see decimals. This may suggest it is not a continuous variable. Read the dataset description: https://archive.ics.uci.edu/ml/datasets/wine+quality
It is already suggested there: "Output variable (based on sensory data): 12 - quality (score between 0 and 10)". It is the only variable that is not based on physicochemical tests. It was probably a subjective score defined by wine tasters.

If you build a regression model to explain wine quality, your research question should sound: "What determines perception of the quality of wine??" to be very precise. Do you know why?

In [ ]:
wine.mean()

What do mean results tell us? Here they give a quick overview, suggesting that variables are in different ranges. We know that quality variable is a score between 0 and 10. Let's see what are the minimum and maximum values for the physicochemical variables:

In [ ]:
wine.min()

In [ ]:
wine.max()

### Check your variables

Use lmplot() function from Seaborn to explore linear relationship 
Input data must be in a Pandas DataFrame. To plot them, we provide the predictor and response variable names along with the dataset

In [ ]:
sns.lmplot(wine, x = "quality", y = "density", aspect=2);

Did you find outliers or missing data? 
You can use function np.unique and find the unique elements of an array.

In [ ]:
?np.unique

In [ ]:
for column in wine.columns:
    unique_values = wine[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

Now we can see clearly what we already suspected: the variables have different value ranges i.e. are on different scales. 
Also, it looks like that not the full quality scale is actually used in the data. The lowest score is 3, the highest 8. The total_sulfur_dioxide has values of even 289, whereas citric_acid is maximum 1. The latter may suggest it is ratio or %. But alcohol for sure is expressed in %, but the max value is only 14.9. 
What shall we do then...? Looks like standarization is needed. Or normalization?

For that we will use a short code written by de Filippi. Check also his explanation of normalization and standarisation.
https://medium.com/@rrfd/standardize-or-normalize-examples-in-python-e3f174b65dfc

In [ ]:
# 1D Visualizations
plt.figure(figsize=(9, 5)) # Setting the figure's size: format width, height (in inches)
plt.subplot(1,2,1) # subplot(nrows, ncols, index, **kwargs)
plt.boxplot(wine.quality)
plt.title("Boxplot of the Quality attribute")
plt.subplot(1,2,2)
plt.boxplot(wine.density)
plt.title("Boxplot of the Density attribute")

In [ ]:
# 2D Visualization
plt.scatter(wine.quality, wine.density, alpha = .5)
plt.xlabel('Wine Quality')
plt.ylabel('Wine Density')
plt.title('Wine Quality vs. Density\n');

Do you need to remove any cases?

In [ ]:
# calculate the IQR for density
wine_density_iqr = wine['density'].quantile(0.75) - wine['density'].quantile(0.25)
# calculate values below Q1 
lb_wine_density = wine['density'].quantile(0.25) - 1.5 * wine_density_iqr
# calculate values above Q3
ub_wine_density = wine['density'].quantile(0.75) + 1.5 * wine_density_iqr
wine_cleaned_density = wine[(wine['density'] >= lb_wine_density) & (wine['density'] <= ub_wine_density)]

In [ ]:
# calculate the IQR for quality
wine_quality_iqr = wine_cleaned_density['quality'].quantile(0.75) - wine_cleaned_density['quality'].quantile(0.25)
# calculate values below Q1 
lb_wine_quality = wine_cleaned_density['quality'].quantile(0.25) - 1.5 * wine_quality_iqr
# calculate values above Q3
ub_wine_quality = wine_cleaned_density['quality'].quantile(0.75) + 1.5 * wine_quality_iqr
wine_cleaned = wine_cleaned_density[(wine_cleaned_density['quality'] >= lb_wine_quality) & (wine_cleaned_density['quality'] <= ub_wine_quality)]

In [ ]:
# 1D Visualizations
plt.figure(figsize=(9, 5)) # Setting the figure's size: format width, height (in inches)
plt.subplot(1,2,1) # subplot(nrows, ncols, index, **kwargs)
plt.boxplot(wine_cleaned.quality)
plt.title("Boxplot of the Quality attribute")
plt.subplot(1,2,2)
plt.boxplot(wine_cleaned.density)
plt.title("Boxplot of the Density attribute")

Did you need to standarize data?

In [ ]:
#This is code for standarization  
from sklearn import preprocessing
import numpy as np

#Get column names first
names = wine.columns
#Create the Scaler object
scaler = preprocessing.StandardScaler()
#Fit your data on the scaler object
st_wine = scaler.fit_transform(wine)
st_wine = pd.DataFrame(st_wine, columns=names)

Let's run some histograms below to check the distribution *after* the standarization. That's why we use the new data frame called "st_wine". Compare with "wine" only dataset and see what changed.

In [ ]:
ax = sns.boxplot(data=st_wine, orient="h")

In [ ]:
st_wine.hist()

You can see that some of variables are skewed. The histogram for 'wine quality' confirms it is not a continuos variable. 

## Form ideas about the data 
Before you move on to exploring correlations and maybe other kinds of models, try and build some sense of understanding of the relations between the variables. What are some relations that stand out. Do you know a bit more about the wines in this dataset or wines more generally?

## Move on to building some simple models

You can calculates a Pearson correlation coefficient and the p-value for testing non-correlation.

We will be using the scikit-learn package here. This is a package we will be making use of very frequently.

In [ ]:
#| error: true
import scipy.stats
scipy.stats.pearsonr(st_wine.alcohol.values, st_wine.density.values)

using **Scikit-learn**, build a simple linear regression (OLS) 

In [ ]:
#| error: true
from sklearn.linear_model import LinearRegression

est = LinearRegression(fit_intercept = True)

x = st_wine[['alcohol']]
y = st_wine[['density']]

est.fit(x, y)

print("Coefficients:", est.coef_)
print ("Intercept:", est.intercept_)

What is the model's mean squared error ($MSE$) and the coefficient of determination ($R^2$) ?

In [ ]:
#| error: true
from sklearn import metrics

model = LinearRegression()
model.fit(x, y)
y_hat = model.predict(x)
plt.plot(x, y,'o', alpha = 0.5)
plt.plot(x, y_hat, 'r', alpha = 0.5)
plt.xlabel('alcohol')
plt.ylabel('density')
print ("MSE:", metrics.mean_squared_error(y, y_hat))
print ("R^2:", metrics.r2_score(y, y_hat))
print ("var:", y.var())
plt.savefig("wine.png", dpi = 300, bbox_inches = 'tight')

What's the conclusion? 
The regression line shows a linear relationship. We can see a negative trend. The more alcohol in the wine, the less density wine has.  

We aim for **MSE** to be low, the closer to 0 the better. It's much lower than in our icesea example.

And *$R^2$* is a bit low (0.24), given that it can have values between 0 and 1. It indicates what portion of the variance of wine density was explained - around 24%. That's why we can try adding more predictors to see if they help explain the density of wine. 
Also, in single linear regression you can use a square of correlation coefficient for variance explained: 0.49*0.49 (see correlation result above) equals 0.24 ($R^2$).

The **Intercept** tells us (the constant) is the expected mean value of Y when all X=0. In our case we have only one x - alcohol. 
How  do we interpret it?

*standarized values*: when there is no alcohol in wine, the density will be lower by -3.4 standard deviations.

*not standarized values*: when there is no alcohol in wine, the density equals 1.

**Coefficient** equals -0.496 (based on standarized values), i.e. if value of alcohol increases by 1 standard deviation, then density will decrease by 0.49 of its standard deviation. In non-standarized values it equals -0.0008, so if alcohol is increased by 1%, the density will decrease by... Exactly, how much density is a lot...? We don't know that. It's harder to interpret than e.g. 1 cm or 1 year. That's where standarized values come handy.


## Multiple linear regression
 

Now we can try adding more predictors to see if they help explain the density of wine.

In [ ]:
from sklearn.linear_model import LinearRegression

est = LinearRegression(fit_intercept = True) 
x = st_wine[['alcohol','fixed_acidity']]
y = st_wine[['density']]

est.fit(x, y)

print("Coefficients:", est.coef_)
print ("Intercept:", est.intercept_)


from sklearn import metrics

model = LinearRegression()
model.fit(x, y)
y_hat = model.predict(x)
print ("MSE:", metrics.mean_squared_error(y, y_hat))
print ("R^2:", metrics.r2_score(y, y_hat))
print ("var:", y.var())

We can see that by adding acidity our R2 got much better. Now we continue the fun by trying to find even better model.